Here's what I will do. 

For precision n

I'll map an inp to activation layer lying in the range x , x + 2^(-n) to a value y. Both x and y will have precision 2^(-n). How to decide upon 'y' ? 

We will choose that y which is closest  to avg of AF(x) and AF(x + 2^-n)

In [ ]:
import six
import numpy as np
import tensorflow.compat.v2 as tf
from tensorflow.keras.layers import *
from tensorflow.keras.models import Model
from tensorflow.keras.datasets import mnist
from tensorflow.keras.utils import to_categorical
import matplotlib.pyplot as plt
import keras

!pip install git+https://github.com/google/qkeras

In [ ]:
import tensorflow as tf

from tensorflow.keras import datasets, layers, models
import matplotlib.pyplot as plt

from qkeras import *

(train_images, train_labels), (test_images, test_labels) = datasets.mnist.load_data()

# Normalize pixel values to be between 0 and 1
train_images, test_images = train_images / 255.0, test_images / 255.0

def get_one_hot(targets, nb_classes):
    res = np.eye(nb_classes)[np.array(targets).reshape(-1)]
    return res.reshape(list(targets.shape)+[nb_classes])
#train_labels, test_labels = get_one_hot(train_labels, 10), get_one_hot(test_labels, 10)#

train_images = train_images.reshape(train_images.shape + (1,)).astype("float32")
test_images = test_images.reshape(test_images.shape + (1,)).astype("float32")

In [ ]:
import tensorflow as tf

from tensorflow.keras import datasets, layers, models
import matplotlib.pyplot as plt

from qkeras import *

In [ ]:
(train_images, train_labels), (test_images, test_labels) = datasets.cifar10.load_data()

# Normalize pixel values to be between 0 and 1
train_images, test_images = train_images / 255.0, test_images / 255.0

def get_one_hot(targets, nb_classes):
    res = np.eye(nb_classes)[np.array(targets).reshape(-1)]
    return res.reshape(list(targets.shape)+[nb_classes])

# Tanh Intervals and Mappings

## 8 bit

In [ ]:
def swish(x):
  #global intervals, coeffArray
  #return x
  #coeff = np.array(coeffArray)
  #conditionArray = sum([tf.multiply(tf.cast(tf.math.logical_and(tf.math.less(x, 0.03125*(n+1)), tf.math.greater_equal(x, 0.03125*n)), tf.float32), coeff[n][0]*x + coeff[n][1]*K.ones_like(x)) for n in range(256)])

  #return quantized_bits(17, 1)(conditionArray)
  #return conditionArray

  quantizer = quantized_bits(precision+1, 1)
  xprime = quantizer(x)
  argument = 0.5*(tf.math.sigmoid(xprime) + tf.math.sigmoid(xprime + 2**(-precision)))
  return quantizer(argument)

In [ ]:
intBits = 1
precision = 32

def modelMaker(precision, intBits):
  model = models.Sequential()
  #model.add(layers.Conv2D(32, (3, 3), activation='sigmoid', input_shape=(32, 32, 3)))
  model.add(QConv2D(32, (3,3), kernel_quantizer="quantized_bits({}, {} , alpha=1)".format(precision, intBits),
          bias_quantizer="quantized_bits({}, {} , alpha=1)".format(precision, intBits),
          activation='sigmoid',
          name='c1'))
  model.add(QActivation('quantized_bits({}, {})'.format(precision, intBits)))
  model.add(layers.MaxPooling2D((2, 2)))
  #model.add(layers.Conv2D(64, (3, 3), activation='sigmoid'))
  model.add(QConv2D(64, (3,3), kernel_quantizer="quantized_bits({}, {} , alpha=1)".format(precision, intBits),
          bias_quantizer="quantized_bits({}, {} , alpha=1)".format(precision, intBits),
          activation='sigmoid',
          name='c2'))
  model.add(QActivation('quantized_bits({}, {})'.format(precision, intBits)))

  model.add(layers.MaxPooling2D((2, 2)))
  #model.add(layers.Conv2D(64, (3, 3), activation='sigmoid'))
  model.add(QConv2D(64, (3,3), kernel_quantizer="quantized_bits({}, {} , alpha=1)".format(precision, intBits),
          bias_quantizer="quantized_bits({}, {} , alpha=1)".format(precision, intBits),
          activation='sigmoid',
          name='c3'))
  model.add(QActivation('quantized_bits({}, {})'.format(precision, intBits)))

  model.add(layers.Flatten())
  #model.add(layers.Dense(64, activation='sigmoid'))
  model.add(QDense(64, kernel_quantizer="quantized_bits({}, {} , alpha=1)".format(precision, intBits),
          bias_quantizer="quantized_bits({}, {} , alpha=1)".format(precision, intBits),
          name='d1'))
  model.add(Activation(swish))
  model.add(QDense(10, kernel_quantizer="quantized_bits({}, {} , alpha=1)".format(precision, intBits),
          bias_quantizer="quantized_bits({}, {} , alpha=1)".format(precision, intBits),
          name = 'd2'))
  
  return model

In [ ]:
import qkeras
from qkeras import *
from qkeras.utils import *

In [ ]:
qmodels = []
histories = []
for i in [9, 13, 17, 33]:
  precision = i
  for epoch in range(5, 10):
    qmodel = modelMaker(precision, 1)
    qmodel.compile(optimizer='adam',
                  loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
                  metrics=['accuracy'])

    history = qmodel.fit(train_images[:48000], train_labels[:48000], epochs= epoch,
                  validation_data=(train_images[48000:], train_labels[48000:]), verbose=False)
    
    model_save_quantized_weights(qmodel)

    qmodels.append(qmodel)
    histories.append(history)
    qmodel.evaluate(test_images, test_labels)
  

In [ ]:
model.evaluate(test_images[5000:], test_labels[5000:])

In [ ]:
qmodels = []
histories = []
for i in [25]:
  precision = i
  for epoch in range(3, 15):
    qmodel = modelMaker(precision, 1)
    qmodel.compile(optimizer='adam',
                  loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
                  metrics=['accuracy'])

    history = qmodel.fit(train_images[:48000], train_labels[:48000], epochs= epoch,
                  validation_data=(train_images[48000:], train_labels[48000:]), verbose=False)
    
    model_save_quantized_weights(qmodel)

    qmodels.append(qmodel)
    histories.append(history)
    qmodel.evaluate(test_images, test_labels)
  